In [1]:
pip install langchain openai requests


In [2]:
your_api_key=""

# Creating a simple weather tool

In [3]:
from langchain.tools import BaseTool
import requests

In [4]:
from langchain.tools import BaseTool
import requests

class WeatherTool(BaseTool):
    name: str = "WeatherChecker"
    description: str = "Use this tool to get the current weather for a city."

    def _run(self, city: str):
        # Example: Using Open-Meteo API (free, no API key)
        url = f"https://api.open-meteo.com/v1/forecast?latitude=28.6139&longitude=77.2090&current_weather=true"
        response = requests.get(url).json()
        temp = response["current_weather"]["temperature"]
        weather_code = response["current_weather"]["weathercode"]
        umbrella = "Yes" if weather_code in [61, 63, 65, 80, 81, 82] else "No"
        return f"Temperature: {temp}°C, Should carry umbrella? {umbrella}"

    async def _arun(self, city: str):
        raise NotImplementedError("Async not implemented")

INITIALIZE AN AGENT

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool

# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=your_api_key)

# Wrap the tool
tools = [Tool(
    name="WeatherChecker",
    func=lambda city: WeatherTool()._run(city),
    description="Get current weather for a city"
)]

# Initialize agent
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",  # Uses reasoning + acting loop
    verbose=True
)

/tmp/ipython-input-2557622654.py:15: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
pip install langchain_community

ASKING THE AGENT

In [10]:
question = "What is the current weather in Haveri and should I carry an umbrella?"
response = agent.run(question)
print(response)




> Entering new AgentExecutor chain...
I should check the current weather in Haveri to see if I need to carry an umbrella.
Action: WeatherChecker
Action Input: Haveri
Observation: Temperature: 28.8°C, Should carry umbrella? No
Thought:I have the current weather information for Haveri and I don't need to carry an umbrella.
Final Answer: The current weather in Haveri is 28.8°C and I do not need to carry an umbrella.

> Finished chain.
The current weather in Haveri is 28.8°C and I do not need to carry an umbrella.
